# TP Mode Selection and Cross Validation

### author: Anastasios Giovanidis, 2019-2020

This is the TP related to mode selection and cross-validation. We will focus here on regression only, but the techniques can be generalised for classification.
We will need to import the following libraries.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import random

In [2]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

## Exercise 1  (Polynomial Regression)

We will now work on problems of polynomial regression. As a means of example, we will try to fit data sampled from the following curve:

$y = 4 + 2x + 0.5x^2 - 0.07x^3 + \epsilon$.

First create synthetic data for $(x_1,x_2,x_3,y,Er)$. The regression plane (>2 dimensions) is given by the following expression, for $\ell$ unknown features:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x + \hat{\beta}_2x^2 +\ldots + \hat{\beta}_{\ell}x^{\ell}$.

**We will use again a multi-linear regression model, to estimate the unknowns.** 
We propose to use regression models with different polynomial degree, because in reality we do not know which is the appropriate model. 

**Question:**
* We start with a linear model of degree $\ell=3$. Split the total dataset of $n=100$ in a train set and a test set. Plot the least-squares curve and output the $R^2$ value. (use $sigmae = 5$, $xl=0, xh = 10$, $n=200$)

* Repeat the process for $\ell = 1, 2, 3, 6, 9, 16$. 

(a) plot the $MSE$ value versus $\ell$ for the Train data. 
(b) plot the $MSE$ value versur $\ell$ for the Test data. What do you observe?

(c) plot the $R^2$ value versus $\ell$ for the Train data. 
(d) plot the $R^2$ value versur $\ell$ for the Test data. What do you observe?

**Build synthetic data**

In [3]:
n = 100
b0 = 4
b1 = np.array([2,0.5,-0.07])
mue, sigmae = 0, 5
xl, xh = 0, 10
np.random.seed(199)
Er = np.random.normal(mue, sigmae, n)
np.random.seed(199)
x0 = np.random.uniform(xl,xh,n)
x = np.array([x0])
x = np.append(x,np.array([x0**2]),axis=0)
x = np.append(x,np.array([x0**3]),axis=0)
#
y = b0 + b1[0]*x[0]+ b1[1]*x[1]+ b1[2]*x[2]+Er

**Solution:**

## Exercise 2 - Resampling

Find the average $MSE_{test}$ for the data sets:

$D_4 = \left\{(1,3),\ (2,4),\ (3,8),\ (4,9) \right\}$.

$D_6 = \left\{(1,3),\ (2,4),\ (3,8),\ (4,9), \ (5,12),\ (7,14) \right\}$.

- For these use 3-fold CV method (same as LOOCV).

- Repeat with the 2-fold CV method, for the $MSE_{test}$.

- Repeat with the Bootstrap method to derive the estimates $(\hat{\beta}_0,\ \hat{\beta}_1)$ and their Standard Error (SE).

Part II (**bonus**): Apply the same method to the data set below for the TV-sales pair:

In [4]:
directory = "/Users/Fishbone/Dropbox/DataNets/ISLbook/Datasets/"
prefix = "Advertising.csv"
filename1 = directory+prefix
dataAd = np.loadtxt(filename1, delimiter=",",skiprows=1,usecols=[1,2,3,4])

In [5]:
# pandas
pdAd = pd.DataFrame(dataAd, columns=["TV","radio","newspaper","sales"])

In [6]:
TV = pdAd.iloc[:,0].values
#radio = pdAd.iloc[:,1].values
#news = pdAd.iloc[:,2].values
sales = pdAd.iloc[:,3].values

**Answer:**

How to generate all possible LOOCV data splits:

In [7]:
from sklearn.model_selection import LeaveOneOut
X = np.array([1,2,3,4])
y = np.array([3,4,8,9])
loo = LeaveOneOut()
loo.get_n_splits(X)
print('Number of splits =',loo.get_n_splits(X))
for train_index, test_index in loo.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("TRAIN:", "X=", X_train, "y=", y_train, "TEST:", "X=", X_test, "y=",  y_test)

Number of splits = 4
TRAIN: X= [2 3 4] y= [4 8 9] TEST: X= [1] y= [3]
TRAIN: X= [1 3 4] y= [3 8 9] TEST: X= [2] y= [4]
TRAIN: X= [1 2 4] y= [3 4 9] TEST: X= [3] y= [8]
TRAIN: X= [1 2 3] y= [3 4 8] TEST: X= [4] y= [9]


How to generate all possible k-folds CV data splits? We do not have to implement k-fold cross-validation manually. The scikit-learn library provides an implementation that will split a given data sample up.

The KFold() scikit-learn class can be used. It takes as arguments the number of splits, whether or not to shuffle the sample, and the seed for the pseudorandom number generator used prior to the shuffle.

For example, we can create an instance that splits a dataset into 3 folds, shuffles prior to the split, and uses a value of 1 for the pseudorandom number generator.

In [8]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)
kf.get_n_splits(X)
print('Number of splits =',kf.get_n_splits(X)) 
#KFold(n_splits=2, shuffle=True, random_state=1)
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print("TRAIN:", "X=", X_train, "y=", y_train, "TEST:", "X=", X_test, "y=",  y_test)

Number of splits = 3
TRAIN: X= [3 4] y= [8 9] TEST: X= [1 2] y= [3 4]
TRAIN: X= [1 2 4] y= [3 4 9] TEST: X= [3] y= [8]
TRAIN: X= [1 2 3] y= [3 4 8] TEST: X= [4] y= [9]


How to use the cross_validate routine

In [9]:
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_validate
#diabetes = datasets.load_diabetes()
#X = diabetes.data[:150]
#y = diabetes.target[:150]
#
# Dataset example with n=4
X = np.array([1,2,3,4])
X_sc = X[:,np.newaxis]
y = np.array([3,4,8,9])
regr = linear_model.LinearRegression()

In [10]:
cv_results = cross_validate(regr, X_sc, y, cv=3, scoring=('r2', 'neg_mean_squared_error'), return_train_score=False)
print(sorted(cv_results.keys()))

['fit_time', 'score_time', 'test_neg_mean_squared_error', 'test_r2']


/Users/Fishbone/anaconda3/lib/python3.6/site-packages/sklearn/metrics/regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/Users/Fishbone/anaconda3/lib/python3.6/site-packages/sklearn/metrics/regression.py:543: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [11]:
print(cv_results['test_r2'])

[-35.  nan  nan]


In [12]:
print(cv_results['test_neg_mean_squared_error'])

[-9.         -1.65306122 -1.        ]
